In [65]:
#前提
import pandas as pd
import numpy as np

In [66]:
#学習ファイル入力
df_train_start = pd.read_csv("train.csv")

In [4]:
import pandas as pd
df_train_start = pd.read_csv("train.csv")
df_weather = pd.read_csv("weather.csv")
print(df_train_start.head())
print(df_weather.head())


   kind      date  amount  mode_price      area  year  weekno
0  だいこん  20051107  201445       735.0        千葉  2005      45
1  だいこん  20051108  189660       840.0  千葉_各地_青森  2005      45
2  だいこん  20051110  218166       735.0  千葉_各地_青森  2005      45
3  だいこん  20051111  182624       682.5     千葉_青森  2005      45
4  だいこん  20051112  220691       682.5     千葉_青森  2005      45
       date  mean_temp  max_temp     max_temp_time  min_temp  \
0  20041106       13.1      20.1   2004/11/6 12:50       8.1   
1  20041107        9.6      12.5   2004/11/7 13:40       5.4   
2  20041108        9.0      15.5   2004/11/8 12:51       3.2   
3  20041109       12.2      17.7   2004/11/9 14:40       5.7   
4  20041110       11.6      16.4  2004/11/10 14:42       6.9   

      min_temp_time  sum_rain  sun_time  mean_humid area  
0   2004/11/6 05:31       5.0       1.6        77.0   青森  
1   2004/11/7 22:17       0.0       4.1        63.0   青森  
2   2004/11/8 06:28       0.0       8.9        72.0   青森  
3   200

In [67]:
#資料の加工関数
def data_pre(df_vegetable):
  #天気データの入力
  df_weather = pd.read_csv("weather.csv")

  #インデックスを適当な数字に
  df_weather.reset_index(inplace = True)
  df_vegetable.reset_index(inplace = True)

  ##野菜
  #野菜を11種類に
  df_vegetable = df_vegetable[df_vegetable['kind'].isin(['だいこん','こまつな','ほうれんそう','ねぎ','きゅうり','じゃがいも','とうもろこし','いんげん','えだまめ','レイシにがうり','ミニトマト'])]
  #野菜の使わない列削除
  df_vegetable = df_vegetable.drop('weekno',axis = 1)
  if 'amount'in df_vegetable.columns:
    df_vegetable = df_vegetable.drop('amount',axis = 1)
  # # #野菜の年月日作成
  # df_vegetable['date'] = pd.to_datetime(df_vegetable['date'].astype(int), format='%Y%m%d')
  # # #野菜の5か月前までの年月作成
  # for i in range(1,6):
  #   df_vegetable[f'date{i}'] = df_vegetable['date'] + pd.DateOffset(months=-i)
  #   df_vegetable[f'date{i}'] = df_vegetable[f'date{i}'].dt.strftime('%Y%m%d').astype(int)
  # df_vegetable['date'] = df_vegetable['date'].dt.strftime('%Y%m%d').astype(int)


  #野菜のareaをwetherの表示とそろえる
  df_vegetable["area"]  = df_vegetable["area"].apply(lambda x: next((p for p in str(x).split("_")  if p != '各地'), "東京"))

  replace_area = {
      "神奈川":"横浜",
      "茨城":"水戸",
      "北海道":"帯広",
      "埼玉":"熊谷",
      "群馬":"前橋",
      "岩手":"盛岡",
      "栃木":"宇都宮",
      "中国":"高松",
      "静岡":"浜松",
      "山梨":"甲府",
      "沖縄":"那覇",
      "愛知":"名古屋"
  }

  df_vegetable['area'] = df_vegetable['area'].replace(replace_area, regex=False)

  ##天気
  #天気の使わない行削除、野菜にない地域、temp、temp_time
  df_weather = df_weather[df_weather['area'].isin(df_vegetable['area'])]
  weather_drop = {"max_temp", "min_temp", 'max_temp_time', "min_temp_time", "sun_time"}
  df_weather = df_weather.drop(weather_drop,axis=1)

  #月ごとに天気データの平均をとり翌月に代入

  #天気の月平均データ入れる
  columns = ["mean_temp",	"sum_rain",	"mean_humid"]
  months = [[1,30],[2,60],[3,90],[4,120],[5,150]]
  df_weather['date'] = pd.to_datetime(df_weather['date'].astype(int), format='%Y%m%d')
  dfs = []
  for area, group in df_weather.groupby('area'):
    group = group.sort_values('date')  # 時系列に並べる
    group = group.set_index("date")    #インデックスに変える
    for month, days in months:
      for col in columns:
          group[f"month_{col}_{month}"] = (
              group[col]
              .rolling(f'{days}D',min_periods=5)  # 過去30日間の平均
              .mean()
          )
    group = group.reset_index()
    min_valid_date = group["date"].min() + pd.DateOffset(days=180)
    group = group[group["date"] >= min_valid_date]
    dfs.append(group)
  df_weather = pd.concat(dfs)
  df_weather['date'] = df_weather['date'].dt.strftime('%Y%m%d').astype(int)

  #天気の計算後使わない要素を削除
  for k in columns:
    df_weather = df_weather.drop(k,axis=1)

  ##データ合体
  # 天気を野菜と月ごとにつなげる
  df = pd.merge(df_vegetable, df_weather, on=['date', 'area'], how='left')

  # for i in range(1,6):
  #   df_weather_change = df_weather.rename(columns={'date': f'date{i}'})
  #   weather_cols_to_rename = df_weather_change.columns.difference([f'date{i}', 'area'])

  #   df = pd.merge(df, df_weather_change, on=[f'date{i}', 'area'], how='left')
  #   df.rename(columns={col: f"{col}_{i}" for col in weather_cols_to_rename}, inplace=True)
  #   df = df.drop( f'date{i}',axis=1)

  #インデックスをkindとdateにし、データ合体後使わないarea削除
  df.set_index(["kind", "date"], inplace=True)
  df_drop = {'index_x', 'area', 'index_y'}
  df = df.drop(df_drop,axis=1, errors="ignore")

  return df


In [68]:
df = data_pre(df_train_start)

In [69]:
df

mode_price  year  month_mean_temp_1  month_sum_rain_1  \
kind  date                                                              
だいこん  20051107       735.0  2005          17.413333          5.500000   
      20051108       840.0  2005          17.373333          4.950000   
      20051110       735.0  2005          17.090000          4.266667   
      20051111       682.5  2005          16.886667          4.266667   
      20051112       682.5  2005          16.703333          4.616667   
...                    ...   ...                ...               ...   
ミニトマト 20230725       184.0  2023          27.983333          1.200000   
      20230727       184.0  2023          28.310000          1.200000   
      20230728       108.0  2023          28.420000          1.200000   
      20230729       108.0  2023          28.510000          1.100000   
      20230731       108.0  2023          28.823333          0.550000   

                month_mean_humid_1  month_mean_temp_2  month_sum_rain_2  \
kind  date                                                                
だいこん  20051107           76.300000          20.178333          4.075000   
      20051108           74.966667          20.061667          4.075000   
      20051110           72.900000          19.653333          3.983333   
      20051111           72.933333          19.425000          3.975000   
      20051112           72.600000          19.198333          4.150000   
...                            ...                ...               ...   
ミニトマト 20230725           74.300000          25.100000          6.641667   
      20230727           73.133333          25.413333          6.641667   
      20230728           72.766667          25.590000          6.333333   
      20230729           72.400000          25.733333          6.333333   
      20230731           70.933333          26.081667          6.283333   

                month_mean_humid_2  month_mean_temp_3  month_sum_rain_3  \
kind  date                                                                
だいこん  20051107           76.100000          22.397778          5.750000   
      20051108           75.700000          22.295556          5.716667   
      20051110           75.000000          22.010000          5.672222   
      20051111           74.933333          21.857778          5.538889   
      20051112           74.666667          21.700000          5.655556   
...                            ...                ...               ...   
ミニトマト 20230725           76.833333          22.951111          6.000000   
      20230727           76.600000          23.236667          6.000000   
      20230728           76.133333          23.358889          5.988889   
      20230729           75.783333          23.473333          5.955556   
      20230731           75.700000          23.760000          5.944444   

                month_mean_humid_3  month_mean_temp_4  month_sum_rain_4  \
kind  date                                                                
だいこん  20051107           76.444444          23.440000          5.025000   
      20051108           76.077778          23.360000          5.025000   
      20051110           75.600000          23.245000          5.020833   
      20051111           75.477778          23.161667          5.016667   
      20051112           75.333333          23.057500          5.104167   
...                            ...                ...               ...   
ミニトマト 20230725           74.788889          21.097500          5.375000   
      20230727           74.877778          21.420833          5.220833   
      20230728           74.911111          21.556667          5.216667   
      20230729           74.655556          21.681667          5.216667   
      20230731           74.944444          21.939167          5.216667   

                month_mean_humid_4  month_mean_temp_5  month_sum_rain_5  \
kind  date                                                     

In [70]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

#学習に使う項目の整理
analysis = {
    'だいこん':["month_mean_temp_4", "month_mean_temp_5"],
    'こまつな':["month_mean_temp_2","month_mean_humid_2", "month_sum_rain_2", "month_mean_temp_3","month_mean_humid_3", "month_sum_rain_3"],
    'ほうれんそう':["month_mean_temp_1","month_mean_humid_1", "month_mean_temp_2","month_mean_humid_2"],
    'ねぎ':["month_mean_temp_4","month_mean_humid_4", "month_mean_temp_5","month_mean_humid_5"],
    'きゅうり':[ "month_mean_temp_4","month_mean_humid_4", "month_mean_temp_5","month_mean_humid_5"],
    'じゃがいも':["month_mean_temp_3", "month_mean_temp_4"],
    'とうもろこし':["month_mean_temp_2","month_mean_humid_2", "month_mean_temp_3","month_mean_humid_3"],
    'いんげん':["month_sum_rain_2", "month_sum_rain_5"],
    'えだまめ':["month_mean_temp_2","month_mean_humid_2", "month_mean_temp_3","month_mean_humid_3", "month_mean_temp_4","month_mean_humid_4"],
    'レイシにがうり':[ "month_mean_temp_1", "month_mean_temp_2", "month_mean_temp_3"],
    'ミニトマト':[ "month_mean_temp_3","month_mean_humid_3", "month_mean_temp_4","month_mean_humid_4"]
}
vegs = ['だいこん', 'こまつな', 'ほうれんそう', 'ねぎ', 'きゅうり', 'じゃがいも', 'とうもろこし', 'いんげん','えだまめ', 'レイシにがうり', 'ミニトマト']
score = []

#学習してモデル作成後に即予測して保存
for veg in vegs:

  df_veg = df.loc[veg]
  features = analysis[veg]
  col = "mode_price"

  #野菜ごとにtrainとvalに分けて処理
  df_train, df_val = train_test_split(df_veg, test_size=0.2)
  train_y = df_train[col]
  train_x = df_train[features].fillna(0)
  val_y = df_val[col]
  val_x = df_val[features].fillna(0)
  trains = lgb.Dataset(train_x, train_y)
  valids = lgb.Dataset(val_x, val_y)

  #モデルの作成
  params ={
    "objective": "regression",   #数値回帰
    "metrics": "mae",            #平均絶対誤差
    "max_depth": 3,            # 木の深さを控えめに
    "min_data_in_leaf": 10,    # 葉の最小データ数を指定
    "verbosity": -1
  }
  model = lgb.train(params, trains, valid_sets=valids, num_boost_round=500, callbacks=[lgb.early_stopping(stopping_rounds=50)])

  #予測精度
  val_pred = model.predict(val_x)
  mae = mean_absolute_error(val_y, val_pred)
  r2 = r2_score(val_y, val_pred)
  ave = df_veg["mode_price"].mean()
  score.append(f"{veg}\n AVE:{ave:.2f} MAE: {mae:.2f}, R²: {r2:.4f}")

  #テストデータの入力と予測
  df_test = pd.read_csv("test.csv",index_col=0)
  df_test = data_pre(df_test)
  df_test_x = df_test[features].fillna(0)
  predict = model.predict(df_test_x)
  df_test['mode_price'] = predict


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[152]	valid_0's l1: 193.754
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[498]	valid_0's l1: 31.4208
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's l1: 16.1477
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[499]	valid_0's l1: 277.23
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[499]	valid_0's l1: 279.069
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[133]	valid_0's l1: 475.401
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[326]	valid_0's l1: 157.53
Training until validation scores don't improve for 50 rounds
Early stopping, best 

In [71]:
#各野菜の価格の平均を出して、誤差を％表示する


for line in score:
    print(line)


だいこん
 AVE:965.92 MAE: 193.75, R²: 0.2406
こまつな
 AVE:136.46 MAE: 31.42, R²: 0.5920
ほうれんそう
 AVE:114.00 MAE: 16.15, R²: 0.6539
ねぎ
 AVE:1665.55 MAE: 277.23, R²: 0.3791
きゅうり
 AVE:1669.76 MAE: 279.07, R²: 0.5671
じゃがいも
 AVE:2114.49 MAE: 475.40, R²: 0.1044
とうもろこし
 AVE:1257.49 MAE: 157.53, R²: 0.5914
いんげん
 AVE:1782.83 MAE: 340.81, R²: 0.2232
えだまめ
 AVE:282.33 MAE: 51.84, R²: 0.4088
レイシにがうり
 AVE:1030.93 MAE: 164.96, R²: 0.5614
ミニトマト
 AVE:146.34 MAE: 14.64, R²: 0.6430


In [72]:
#csv出力
#df_test["mode_price"].to_csv("submit_test_vegetable2.csv")

In [73]:
# from google.colab import files
# files.download("submit_test_vegetable2.csv")